grasp

In [9]:
import genesis as gs
import numpy as np
import torch

In [2]:
gs.init(backend=gs.gpu, seed=42)


[Genesis] [20:12:28] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [20:12:28] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [20:12:28] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [20:12:29] [INFO] Running on [NVIDIA GeForce RTX 3070 Laptop GPU] with backend gs.cuda. Device memory: 7.78 GB.
[Genesis] [20:12:29] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: 42, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.


In [3]:

scene = gs.Scene(
    show_viewer = True,
    viewer_options = gs.options.ViewerOptions(
        res           = (1280, 960),
        camera_pos    = (3.5, 0.0, 2.5),
        camera_lookat = (0.0, 0.0, 0.5),
        camera_fov    = 40,
        max_FPS       = 60,
    ),
    sim_options=gs.options.SimOptions(
        dt = 0.01,
    ),
    vis_options = gs.options.VisOptions(
        show_world_frame = True,
        world_frame_size = 1.0,
        show_link_frame  = False,
        show_cameras     = False,
        plane_reflection = True,
        ambient_light    = (0.1, 0.1, 0.1),
    ),
    renderer=gs.renderers.Rasterizer(),
)

plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka = scene.add_entity(
    gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
)
cube = scene.add_entity(
    gs.morphs.Box(
        size = (0.04, 0.04, 0.04),
        pos  = (0.65, 0.0, 0.02),
    )
)
gs.morphs.Box(
    size = (0.04, 0.04, 0.04),
    pos  = (0.65, 0.0, 0.02)
)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0, 0, 0.5),
    fov    = 30,
    GUI    = False,
)


[Genesis] [20:12:34] [INFO] Scene <7489c3c> created.
[Genesis] [20:12:34] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <5a75f9c>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [20:12:34] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <b48ae6d>, morph: <gs.morphs.MJCF(file='/home/vahid/envs/panda/panda/lib/python3.10/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [20:12:35] [WARNING] (MJCF) Tendon not supported
[Genesis] [20:12:35] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [20:12:35] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [20:12:35] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [20:12:35] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using 

In [4]:

scene.build()
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

# set control gains
# Note: the following values are tuned for achieving best behavior with Franka
# Typically, each new robot would have a different set of parameters.
# Sometimes high-quality URDF or XML file would also provide this and will be parsed.
franka.set_dofs_kp(
    np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
)
franka.set_dofs_kv(
    np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
)
franka.set_dofs_force_range(
    np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
    np.array([ 87,  87,  87,  87,  12,  12,  12,  100,  100]),
)

# render rgb, depth, segmentation, and normal
# rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)


[Genesis] [20:12:38] [INFO] Building scene <7489c3c>...
[Genesis] [20:12:40] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [20:12:40] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.


[W 04/29/25 20:12:41.461 31884] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [20:12:48] [INFO] Compiling simulation kernels...
[Genesis] [20:14:24] [INFO] Building visualizer...
[Genesis] [20:14:28] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [5]:

# get the end-effector link
end_effector = franka.get_link('hand')

# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = np.array([0.65, 0.0, 0.25]),
    quat = np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos[-2:] = 0.04
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 200, # 2s duration
)

[W 04/29/25 20:14:52.649 31884] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [20:15:03] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.
[Genesis] [20:15:03] [INFO] Ingoring collision pairs already active for starting pos.
[Genesis] [20:15:03] [INFO] Path solution found successfully.


In [38]:
# execute the planned path
for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()


[Genesis] [21:38:16] [INFO] Running at 0.03 FPS.
[Genesis] [21:38:16] [INFO] Running at 0.03 FPS.
[Genesis] [21:38:16] [INFO] Running at 0.03 FPS.
[Genesis] [21:38:16] [INFO] Running at 0.03 FPS.
[Genesis] [21:38:16] [INFO] Running at 0.03 FPS.
[Genesis] [21:38:16] [INFO] Running at 0.03 FPS.
[Genesis] [21:38:16] [INFO] Running at 0.04 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.04 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.04 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.04 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.04 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.05 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.05 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.05 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.05 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.06 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.06 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.06 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.07 FPS.
[Genesis] [21:38:17] [INFO] Running at 0.07 FPS.
[Genesis] [21:38:17]

In [43]:
# reach
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = np.array([0.65, 0.0, 0.130]),
    quat = np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
for i in range(100):
    scene.step()



# grasp
franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([-0.5, -0.5]), fingers_dof)

for i in range(100):
    scene.step()

# lift
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
for i in range(200):
    scene.step()

[Genesis] [00:15:31] [INFO] Running at 4.09 FPS.
[Genesis] [00:15:31] [INFO] Running at 4.29 FPS.
[Genesis] [00:15:32] [INFO] Running at 4.50 FPS.
[Genesis] [00:15:32] [INFO] Running at 4.71 FPS.
[Genesis] [00:15:32] [INFO] Running at 4.93 FPS.
[Genesis] [00:15:32] [INFO] Running at 5.17 FPS.
[Genesis] [00:15:32] [INFO] Running at 5.41 FPS.
[Genesis] [00:15:32] [INFO] Running at 5.67 FPS.
[Genesis] [00:15:32] [INFO] Running at 5.94 FPS.
[Genesis] [00:15:32] [INFO] Running at 6.21 FPS.
[Genesis] [00:15:32] [INFO] Running at 6.50 FPS.
[Genesis] [00:15:32] [INFO] Running at 6.81 FPS.
[Genesis] [00:15:32] [INFO] Running at 7.12 FPS.
[Genesis] [00:15:32] [INFO] Running at 7.45 FPS.
[Genesis] [00:15:32] [INFO] Running at 7.80 FPS.
[Genesis] [00:15:32] [INFO] Running at 8.15 FPS.
[Genesis] [00:15:32] [INFO] Running at 8.52 FPS.
[Genesis] [00:15:32] [INFO] Running at 8.89 FPS.
[Genesis] [00:15:32] [INFO] Running at 9.28 FPS.
[Genesis] [00:15:32] [INFO] Running at 9.70 FPS.
[Genesis] [00:15:32]

In [21]:
# open
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
for i in range(100):
    scene.step()
# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

[Genesis] [20:51:35] [INFO] Running at 0.32 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.34 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.36 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.37 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.39 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.41 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.44 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.46 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.48 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.51 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.53 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.56 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.59 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.62 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.66 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.69 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.73 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.76 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.80 FPS.
[Genesis] [20:51:36] [INFO] Running at 0.84 FPS.
[Genesis] [20:51:36]

[Genesis] [21:31:10] [INFO] Running at 0.06 FPS.


In [24]:
franka.get_dofs_force()

tensor([-0.0103,  0.0048,  0.0108, -0.0005, -0.0040, -0.0004, -0.0068,  0.0004,
        -0.0004], device='cuda:0')

In [39]:
cube_pos = cube.get_pos().cpu().numpy()
cube_quat = cube.get_quat().cpu().numpy()

cube_pos, cube_quat

(array([6.4999902e-01, 2.9570606e-06, 1.9892190e-02], dtype=float32),
 array([-1.3834608e-06,  9.9999994e-01,  4.4306205e-05,  7.9489763e-07],
       dtype=float32))

In [41]:
cube_pos = cube.get_pos().cpu().numpy()
cube_quat = cube.get_quat().cpu().numpy()

qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=cube_pos + np.array([0, 0, 0.3]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
for i in range(100):
    scene.step()
# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

[Genesis] [00:14:45] [INFO] Running at 0.28 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.29 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.31 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.32 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.34 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.36 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.38 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.40 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.42 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.44 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.46 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.48 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.51 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.54 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.56 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.59 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.63 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.66 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.69 FPS.
[Genesis] [00:14:45] [INFO] Running at 0.73 FPS.
[Genesis] [00:14:46]

In [ ]:

cam.start_recording()
import numpy as np

for i in range(120):
    scene.step()
    cam.set_pose(
        pos    = (3.0 * np.sin(i / 60), 3.0 * np.cos(i / 60), 2.5),
        lookat = (0, 0, 0.5),
    )
    cam.render()
cam.stop_recording(save_to_filename='video.mp4', fps=60)